In [1]:
import sympy as sp
from sympy import I, exp, ln

# Define time variable
t = sp.symbols('t', real=True)

# Define the inhomogeneous term u(t)
u = exp(4*t) * sp.Matrix([[3], [-5]])
u = sp.Matrix([[0], [0], [0]])
u = sp.Matrix([[0,0,0]]).transpose()
#u = sp.Matrix([[2, -9]]).transpose()


# Define matrix A
A = sp.Matrix([[0, 1], [-8, -4]])
# A = sp.Matrix([[0, 0, 1], [2, 0, -1],[-1, 0, 0]])
# A = sp.Matrix([[1, 2], [2, 4]])
A = sp.Matrix([[3, sp.Rational(5/2), sp.Rational(-5/2)], [10 , 3, -3],[0,0,0]])
#A = sp.Matrix([[exp(3*t), 2*exp(2*t)], [0, exp(2*t)]]) 

def solve_system_of_first_order_differential_equations(A=None,u=None,x0=None,A_is_fundamental_matrix=False,unit_coor = None):
    tau = sp.symbols('tau', real=True)



    if not A_is_fundamental_matrix:
        # Step 1: Compute the eigenvalue decomposition of A
        P, D = A.diagonalize()

        # set selected coordinate to 1
        if not unit_coor==None:
            for i in range(P.cols): 
                if P[unit_coor, i] != 0:  # Ensure the first element of the eigenvector is non-zero
                    #    Normalize the vector by dividing the entire column
                    P[:, i] = P[:, i] / P[unit_coor, i]
        
        # Step 2: Exponentiate the diagonal matrix D to get e^{Dt}
        D_exp = sp.diag(*[exp(D[i, i] * t) for i in range(D.shape[0])])

        # Step 3: Compute the fundamental matrix (e^{At}) as P * e^{Dt}
        fundamental_matrix = P * D_exp


    else:
        fundamental_matrix = A


    matrix_size = fundamental_matrix.shape[0]
    
    if u == None:
            u = sp.zeros(matrix_size, 1)

    if x0 == None:
        constants = sp.symbols(f'c1:{matrix_size+1}')  # Create c1, c2, ..., cn dynamically
        constant_vector = sp.Matrix(constants).reshape(matrix_size, 1)
        x_homogeneous = fundamental_matrix * constant_vector
    else:
        x_homogeneous = fundamental_matrix*fundamental_matrix.inv().sub(t,0)*x0 

    x_particular = fundamental_matrix*sp.integrate(fundamental_matrix.inv()*u, t)

    x_general = x_homogeneous + x_particular



    return x_general
A = sp.Matrix([[0,3], [9 , -1]]) 
u= exp(4*t)*sp.Matrix([[3, -5]]).transpose()
solve_system_of_first_order_differential_equations(A=A,u=None,unit_coor=0)

Matrix([
[                                            c1*exp(t*(-1/2 + sqrt(109)/2)) + c2*exp(t*(-sqrt(109)/2 - 1/2))],
[c1*exp(t*(-1/2 + sqrt(109)/2))/(1/18 + sqrt(109)/18) + c2*exp(t*(-sqrt(109)/2 - 1/2))/(1/18 - sqrt(109)/18)]])

In [2]:
import sympy as sp
from sympy import exp, I

def solve_system_of_first_order_differential_equations(A=None, u=None, x0=None, A_is_fundamental_matrix=False, unit_coor=None, real=False):
    t = sp.symbols('t', real=True)
    tau = sp.symbols('tau', real=True)

    if not A_is_fundamental_matrix:
        # Step 1: Compute the eigenvalue decomposition of A
        P, D = A.diagonalize()

        # Set selected coordinate to 1
        if unit_coor is not None:
            for i in range(P.cols):
                if P[unit_coor, i] != 0:  # Ensure the first element of the eigenvector is non-zero
                    P[:, i] = P[:, i] / P[unit_coor, i]
        
        # Step 2: Exponentiate the diagonal matrix D to get e^{Dt}
        D_exp = sp.diag(*[exp(D[i, i] * t) for i in range(D.shape[0])])

        # Step 3: Compute the fundamental matrix (e^{At}) as P * e^{Dt}
        fundamental_matrix = P * D_exp
    else:
        fundamental_matrix = A

    matrix_size = fundamental_matrix.shape[0]

    if u is None:
        u = sp.zeros(matrix_size, 1)

    # Step 4: Create the homogeneous and particular solutions
    solution_vectors = []  # To hold individual homogeneous solution vectors

    if x0 is None:
        constants = sp.symbols(f'c1:{matrix_size+1}')  # Create c1, c2, ..., cn dynamically
        constant_vector = sp.Matrix(constants).reshape(matrix_size, 1)
        # Split homogeneous solution into individual vectors
        for i in range(matrix_size):
            # Multiply the i-th column of fundamental matrix with the constant
            homogeneous_vector = constants[i] * fundamental_matrix[:, i]
            collect_list = [exp(sp.Wild('w'))] + [const for const in constants] 
            homogeneous_vector.applyfunc(lambda elem: elem.collect( constants))
            # homogeneous_vector.applyfunc(lambda elem: elem.simplify())
            solution_vectors.append(homogeneous_vector)
    else:
        # Homogeneous solution when initial condition is given
        x_homogeneous = fundamental_matrix * fundamental_matrix.inv().subs(t, 0) * x0
        solution_vectors.append(x_homogeneous)  # Add homogeneous solution based on initial condition

    # Step 5: Compute the particular solution
    x_particular = fundamental_matrix * sp.integrate(fundamental_matrix.inv() * u, t)

    return solution_vectors, x_particular  # Return individual homogeneous vectors and the particular solution

# Example usage
A = sp.Matrix([[0, 0, 1], [2, 0, -1], [-1, 0, 0]]) 
# A = sp.Matrix([[5,3], [9 , -1]]) 
#A = sp.Matrix([[1,2], [2 , 4]]) 
u = sp.Matrix([[7, -2]]).transpose()
x0 = sp.Matrix([[6, 7]]).transpose()
A = sp.Matrix([[exp(2*t),2*exp(3*t)], [0 , exp(3*t)]]) 

homogeneous_solutions, particular_solution = solve_system_of_first_order_differential_equations(A=A, u=u,x0=x0, A_is_fundamental_matrix=True,unit_coor=0, real=False)


# Print the individual homogeneous solutions
for i, sol in enumerate(homogeneous_solutions, 1):
    print(f"Homogeneous Solution {i}:")
    print(sol)
    print()

# Print the particular solution
print("\nParticular Solution:")
print(particular_solution)


Homogeneous Solution 1:
Matrix([[14*exp(3*t) - 8*exp(2*t)], [7*exp(3*t)]])


Particular Solution:
Matrix([[-25/6], [2/3]])


In [3]:
from re import X


fm = sp.Matrix([[exp(3*t), 2*exp(2*t)], [0, exp(2*t)]]) 
u = sp.Matrix([[2, -9]]).transpose()
x0 = sp.Matrix([[5, 6]]).transpose()


x_h = fm*fm.inv().subs(t, 0)*x0 
x_p = sp.integrate(
    fm.inv().subs(t, tau) * u.subs(t, tau), (tau, 0, t))

x_t = x_h + x_p
print(x_t.simplify())


NameError: name 'tau' is not defined

In [223]:
kkkM = sp.Matrix([[2, 9], [-1, -4]])
M.eigenvects()

[(-1,
  2,
  [Matrix([
   [-3],
   [ 1]])])]

In [429]:
import sympy as sp

# Define the matrix
A = sp.Matrix([[1, 1], [-1, 1]])

# Compute the eigenvalues and eigenvectors
eigenvects = A.eigenvects()

# Normalize the eigenvectors so that the first coordinate is 1
normalized_eigenvects = []
for eigenval, multiplicity, eigenvecs in eigenvects:
    normalized_vecs = []
    for eigenvec in eigenvecs:
        if eigenvec[0] != 0:  # Check if the first coordinate is non-zero
            eigenvec = eigenvec / eigenvec[0]  # Normalize so the first coordinate is 1
        normalized_vecs.append(eigenvec)
    normalized_eigenvects.append((eigenval, multiplicity, normalized_vecs))

# Output the normalized eigenvectors
for eigenval, multiplicity, eigenvecs in normalized_eigenvects:
    print(f"Eigenvalue: {eigenval}")
    for eigenvec in eigenvecs:
        print(eigenvec)

Eigenvalue: 1 - I
Matrix([[1], [-I]])
Eigenvalue: 1 + I
Matrix([[1], [I]])


In [260]:
A = sp.Matrix([[1, 2], [2, 4,]])
A.eigenvects()



[(0,
  1,
  [Matrix([
   [-2],
   [ 1]])]),
 (5,
  1,
  [Matrix([
   [1/2],
   [  1]])])]

In [268]:
import sympy as sp
from sympy import exp

# Define time variable and integration variable
t, tau = sp.symbols('t tau', real=True)

# Define the inhomogeneous term u(tau)
u_tau = exp(4 * tau) * sp.Matrix([[3], [-5]])  # Update this for the correct u(tau)
u_tau = sp.Matrix([[0], [0], [0]])  # Replace with the actual inhomogeneous term
u_tau = sp.Matrix([[1], [1]])

# Define matrix A
A = sp.Matrix([[1, 2], [2, 4]])

# Step 1: Compute the eigenvalue decomposition of A
P, D = A.diagonalize()

# Step 2: Exponentiate the diagonal matrix D to get e^{Dt}
D_exp = sp.diag(*[exp(D[i, i] * t) for i in range(D.shape[0])])

# Step 3: Compute the fundamental matrix (e^{At}) as P * e^{Dt}
fundamental_matrix = P * D_exp

# Step 4: Compute the inverse of the fundamental matrix (e^{-At})
inverse_fundamental_matrix = P * sp.diag(*[exp(-D[i, i] * tau) for i in range(D.shape[0])]) * P.inv()

# Step 5: Compute the particular solution
# y_p(t) = e^{At} * ∫ e^{-A*tau} * u(tau) d_tau (integrate from 0 to t)
integral_term = sp.integrate(inverse_fundamental_matrix * u_tau, (tau, 0, t))
y_particular = fundamental_matrix * integral_term

# Step 6: Define the dynamically sized homogeneous solution
matrix_size = fundamental_matrix.shape[0]

# Create a dynamically sized constant vector with symbols c1, c2, ..., cn
constants = sp.symbols(f'c1:{matrix_size+1}')  # Create c1, c2, ..., cn dynamically
constant_vector = sp.Matrix(constants).reshape(matrix_size, 1)

# y_h(t) = fundamental_matrix * constant_vector
y_homogeneous = fundamental_matrix * constant_vector

# Step 7: General solution (homogeneous + particular)
y_general = y_homogeneous + y_particular

# Output the particular solution and the general solution
y_particular_simplified = y_particular.simplify()
y_general_simplified = y_general.simplify()

# Display results
y_general


Matrix([
[ -2*c1 + c2*exp(5*t) - 4*t/5 + (-t/5 + 6/25 - 6*exp(-5*t)/25)*exp(5*t) - 6/25 + 6*exp(-5*t)/25],
[c1 + 2*c2*exp(5*t) + 2*t/5 + 2*(-t/5 + 6/25 - 6*exp(-5*t)/25)*exp(5*t) + 3/25 - 3*exp(-5*t)/25]])

In [283]:
import sympy as sp
from sympy import exp

# Define time variable
t = sp.symbols('t', real=True)

# Define the inhomogeneous term u(t)
u = exp(4*t) * sp.Matrix([[3], [-5]])
u = sp.Matrix([[1], [1]])  # You can set the actual u(t) as required

# Define matrix A
A = sp.Matrix([[1, 2], [2, 4]])

# Step 1: Compute the eigenvalue decomposition of A
P, D = A.diagonalize()

# Step 2: Exponentiate the diagonal matrix D to get e^{Dt}
D_exp = sp.diag(*[exp(D[i, i] * t) for i in range(D.shape[0])])

# Step 3: Compute the fundamental matrix (e^{At}) as P * e^{Dt}
fundamental_matrix = P * D_exp

# Step 4: Compute the inverse of the fundamental matrix (e^{-At})
inverse_fundamental_matrix = P * sp.diag(*[exp(-D[i, i] * t) for i in range(D.shape[0])]) * P.inv()

# Step 5: Compute the particular solution
# y_p(t) = e^{At} * ∫ e^{-A*tau} * u(tau) d_tau (integrate from 0 to t)
tau = sp.symbols('tau', real=True)
integral_term = sp.integrate(inverse_fundamental_matrix.subs(t, tau) * u.subs(t, tau), (tau, 0, t))
y_particular = fundamental_matrix * integral_term

# Step 6: Define the dynamically sized homogeneous solution
matrix_size = fundamental_matrix.shape[0]

# Create a dynamically sized constant vector with symbols c1, c2, ..., cn
constants = sp.symbols(f'c1:{matrix_size+1}')  # Create c1, c2, ..., cn dynamically
constant_vector = sp.Matrix(constants).reshape(matrix_size, 1)

# y_h(t) = fundamental_matrix * constant_vector
y_homogeneous = fundamental_matrix * constant_vector

# Step 7: General solution (homogeneous + particular)
y_general = y_homogeneous + y_particular

# Step 8: Apply the initial condition x(0) = (1, 0)^T
y_general_at_0 = y_general.subs(t, 0)

# Initial condition vector
x0 = sp.Matrix([[1], [0]])

# Step 9: Solve for constants using the initial condition y(0) = x0
constant_solutions = sp.solve(sp.Eq(y_general_at_0, x0), constants)

# Step 10: Substitute the solved constants into the general solution
y_particular_with_ic = y_general.subs(constant_solutions)

# Output the final particular solution
y_particular_simplified = y_particular_with_ic.simplify()

# Display the simplified solution
y_particular_simplified


Matrix([
[   -t*exp(5*t)/5 - 4*t/5 + 11*exp(5*t)/25 + 8/25 + 6*exp(-5*t)/25],
[-2*t*exp(5*t)/5 + 2*t/5 + 22*exp(5*t)/25 - 19/25 - 3*exp(-5*t)/25]])

In [285]:
import sympy as sp
from sympy import exp

# Define time variable
t = sp.symbols('t', real=True)

# Define the inhomogeneous term u(t)
u = exp(4*t) * sp.Matrix([[3], [-5]])
u = sp.Matrix([[1], [1]])  # You can set the actual u(t) as required

# Define matrix A
A = sp.Matrix([[1, 2], [2, 4]])

# Step 1: Compute the eigenvalue decomposition of A
P, D = A.diagonalize()

# Step 2: Exponentiate the diagonal matrix D to get e^{Dt}
D_exp = sp.diag(*[exp(D[i, i] * t) for i in range(D.shape[0])])

# Step 3: Compute the fundamental matrix (e^{At}) as P * e^{Dt}
fundamental_matrix = P * D_exp

# Step 4: Compute the inverse of the fundamental matrix (e^{-At})
inverse_fundamental_matrix = P * sp.diag(*[exp(-D[i, i] * t) for i in range(D.shape[0])]) * P.inv()

# Step 5: Compute the particular solution
# y_p(t) = e^{At} * ∫ e^{-A*tau} * u(tau) d_tau (integrate from 0 to t)
tau = sp.symbols('tau', real=True)
integral_term = sp.integrate(inverse_fundamental_matrix.subs(t, tau) * u.subs(t, tau), (tau, 0, t))
y_particular = fundamental_matrix * integral_term

# Step 6: Define the dynamically sized homogeneous solution
matrix_size = fundamental_matrix.shape[0]

# Create a dynamically sized constant vector with symbols c1, c2, ..., cn
constants = sp.symbols(f'c1:{matrix_size+1}')  # Create c1, c2, ..., cn dynamically
constant_vector = sp.Matrix(constants).reshape(matrix_size, 1)

# y_h(t) = fundamental_matrix * constant_vector
y_homogeneous = fundamental_matrix * constant_vector

# Step 7: General solution (homogeneous + particular)
y_general = y_homogeneous + y_particular

# Step 8: Apply the initial condition x(0) = (1, 0)^T
y_general_at_0 = y_general.subs(t, 0)

# Initial condition vector
x0 = sp.Matrix([[1], [0]])

# Step 9: Solve for constants using the initial condition y(0) = x0
constant_solutions = sp.solve(sp.Eq(y_general_at_0, x0), constants)

# Step 10: Substitute the solved constants into the general solution
y_particular_with_ic = y_general.subs(constant_solutions)

# Output the final particular solution
y_particular_simplified = y_particular_with_ic.simplify()

# Display the simplified solution
y_particular_simplified


Matrix([
[   -t*exp(5*t)/5 - 4*t/5 + 11*exp(5*t)/25 + 8/25 + 6*exp(-5*t)/25],
[-2*t*exp(5*t)/5 + 2*t/5 + 22*exp(5*t)/25 - 19/25 - 3*exp(-5*t)/25]])

In [404]:
from re import X


fm = sp.Matrix([[exp(3*t), 2*exp(2*t)], [0, exp(2*t)]]) 
u = sp.Matrix([[2, -9]]).transpose()
x0 = sp.Matrix([[5, 6]]).transpose()


x_h = fm*fm.inv().subs(t, 0)*x0 
x_p = sp.integrate(
    fm.inv().subs(t, tau) * u.subs(t, tau), (tau, 0, t))

x_t = x_h + x_p
print(x_t.simplify())


Matrix([[-7*exp(3*t) + 12*exp(2*t) + 20/3 - 20*exp(-3*t)/3], [6*exp(2*t) - 9/2 + 9*exp(-2*t)/2]])


In [60]:

from math import e

from sympy.functions.elementary import exponential

#
s = sp.symbols('s')
t = sp.symbols('t', real=True)
n = 3

A = sp.Matrix([[0,1],[-2,-2]])
b = sp.Matrix([[0,2]]).transpose()
d = sp.Matrix([[1,0]])
H = -d*(A-s*sp.eye(2)).inv()*b
print(H)
print(sp.re(H.subs(s, n*I)))
print(sp.im(H.subs(s, n*I)))








Matrix([[2/(s**2 + 2*s + 2)]])
Matrix([[-14/85]])
Matrix([[-12/85]])
